在前面的学习中，我们从同一份样本数据中得到了三个模型，而且三个模型的拟合效果都很好，不过很显然，二次模型和三次模型都过于复杂了，属于过拟合。为了判断哪个模型的性能最优，泛化能力最强，我们将数据集划分为训练集和测试集，然后选择在测试集上表现最好的模型。使用这种方法可以有效的评估模型结果，避免过拟合。

这一节我们将学习一种避免过拟合的新方法：**正则化**（regularization）方法。

在线性回归模型 $ \hat{y} = a_0 + a_1x_1 + a_2x_2 + ... + a_nx_n $ 中，每一个变量 $x_i$ 就是模型的一个特征，参数 $a_i$ 代表相应的特征在模型中所占的权重。在前面的例子里，本来只有一个特征，我们人为造出了两个特征来（$x^2$ 和 $x^3$），得到了三个模型，这两个特征对模型来说其实根本没用，过拟合就是因为模型过于复杂，把这些没用的特征也学习到了。所以如果能降低这些没用特征的权重，譬如直接降到 0，那就不会出现过拟合了，为了达到这一点，我们可以稍微改进下我们的损失函数，对特征的权重进行惩罚。